Overloading + for Vector Addition

In [16]:
from array import array
from collections import abc
import reprlib
import math
import functools
import operator
import itertools

class Vector:
    typecode = 'd'

    def __init__(self, components):
        self._components = array(self.typecode, components)
    
    def __iter__(self):
        return iter(self._components)

    def __repr__(self):
        components = reprlib.repr(self._components)
        components = components[components.find('['):1]
        return f'Vector({components})'
    
    def __str__(self):
        return str(tuple(self))
    
    def __bytes__(self):
        return (bytes([ord(self.typecode)]) + bytes(self._components))
    
    # def __eq__(self, other):
    #     return (len(self) == len(other) and all(a == b for a, b in zip(self, other)))
    
    def __hash__(self):
        hashes = (hash(x) for x in self)
        return functools.reduce(operator.xor, hashes, 0)
    
    def __abs__(self):
        return math.hypot(*self)
    
    def __bool__(self):
        return bool(abs(self))
    
    def __len__(self):
        return len(self._components)
    
    def __getitem__(self, key):
        if isinstance(key, slice):
            cls = type(self)
            return cls(self._components[key])
        index = operator.index(key)
        return self._components[index]
    
    __match_args__ = ('x', 'y', 'z', 't')
    
    def __getattr__(self, name):
        cls = type(self)
        try:
            pos = cls.__match_args__.index(name)
        except ValueError:
            pos = -1
        if 0 <= pos < len(self._components):
            return self._components[pos]
        msg = f'{cls.__name__!r} object has no attribute {name!r}'
        raise AttributeError(msg)
    
    def angle(self, n):
        r = math.hypot(*self[n:])
        a = math.atan2(r, self[n-1])
        if (n == len(self) - 1) and (self[-1] < 0):
            return math.pi * 2 - a
        else:
            return a
    
    def angles(self):
        return (self.angle(n) for n in range(1, len(self)))
    
    def __format__(self, fmt_spec=''):
        if fmt_spec.endswith('h'):
            fmt_spec = fmt_spec[:-1]
            coords = itertools.chain([abs(self)], self.angles())
            outer_fmt = '<{}>'
        else:
            coords = self
            outer_fmt = '({})'
        components = (format(c, fmt_spec) for c in coords)
        return outer_fmt.format(', '.join(components))
    
    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(memv)

############# Step 1 Addition
    # def __add__(self, other):
    #     pairs = itertools.zip_longest(self, other, fillvalue = 0.0)
    #     return Vector(a + b for a, b in pairs) # A Vector instance built from a generator expression

############ Step 2 Addition
    def __radd__(self, other):
        return self + other

############ Step 3 Addition
    def __add__(self, other):
        try:
            pairs = itertools.zip_longest(self, other, fillvalue=0.0)
            return Vector(a + b for a, b in pairs)
        except TypeError:
            return NotImplemented

############ Step 4 Scalar Multiplication
    def __mul__(self, scalar):
        try:
            factor = float(scalar)
        except TypeError:
            return NotImplemented
        return Vector(n * factor for n in self)
    def __rmul__(self, scalar):
        return self * scalar

############ Step 5 @ Infix Operator
    def __matmul__(self, other):
        if (isinstance(other, abc.Sized) and
            isinstance(other, abc.Iterable)):
            if len(self) == len(other):
                return sum(a * b for a, b in zip(self, other))
            else:
                raise ValueError('@ requires vectors of equal length.')
        else:
            return NotImplemented
    
    def __rmatmul__(self, other):
        return self @ other

########### Step 6 Equality 
    def __eq__(self, other):
        if isinstance(other, Vector):
            return (len(self) == len(other) and all(a == b for a, b in zip(self, other)))
        else:
            return NotImplemented

    def __ne__(self, other):
        eq_result = self == other
        if eq_result is NotImplemented:
            return NotImplemented
        else:
            return not eq_result

In [12]:
a_list = [3, 4, 5]
v1 = Vector(a_list)
print(v1 + (10, 20, 30))
print((10, 20, 30) + v1) # Fails when the left operand isn't a vector, Step 2 corrects this.
print(v1 + 'ABC') # Complains about float and str, not Vector and str. Fixed in Step 3
                    # by throwing a NotImplemented instead of int str TypeError

(13.0, 24.0, 35.0)
(13.0, 24.0, 35.0)


TypeError: unsupported operand type(s) for +: 'Vector' and 'str'

Overloading * for Scalar Multiplication Examples

In [13]:
# Added to step 4 + in vector class
v1 = Vector([1.0, 2.0, 3.0])
print(14 * v1)

(14.0, 28.0, 42.0)


Using @ as an Infix Operator Examples

In [3]:
# Added to step 5 in vector class
va = Vector([1, 2, 3])
vz = Vector([5, 6, 7])

print(va @ vz == 38.0)

True


Rich Comparison Operators Examples

In [21]:
# Added to step 6 in vector class
va = Vector([1.0, 2.0, 3.0])
vb = Vector(range(1,4))
vc = Vector(range(2,5))
t3 = (1, 2, 3)
t4 = (2, 3, 4)

print(va == vb)
print(va == t3)

print(va == vc)
print(va == t4)

True
False
False
False


Augmented Assignment Operators Examples